In [ ]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

s = Service('/Users/idohyeong/Documents/chromedriver')
driver = webdriver.Chrome(service=s)
time.sleep(1)

url = 'https://www.naver.com/'
driver.get(url)

query = driver.find_element(by='xpath',value='//*[@id="query"]')
query.send_keys('AI')

btn = driver.find_element(by='xpath',value='//*[@id="sform"]/fieldset/button')
btn.click()

news_tab = driver.find_element(by = 'xpath', value = '//*[@id="lnb"]/div[1]/div/ul/li[3]/a')
news_tab.click()

html = driver.page_source
# print(html)



### 뉴스 기사 URL 수집

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time

# 임시 URL 설정
tmp_url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=AI&sort=2&photo=0&field=0&pd=3&ds=2023.08.01&de=2023.08.01&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20230801to20230801,a:all&start=1'

from urllib.parse import urlparse,parse_qs

# 쿼리 파라미터 추출
params = parse_qs(urlparse(tmp_url).query) 
params # dic

# 페이지 시작 번호 설정
params['start'] = 1 

# 오늘과 지난주 날짜 계산
from datetime import datetime, timedelta
today = datetime.today()
lastweek = today - timedelta(days=7)
today = today.strftime('%Y.%m.%d')
lastweek = lastweek.strftime('%Y.%m.%d')

# 날짜 범위 생성
import pandas as pd
date_list = pd.date_range(lastweek, today) # 지난주~오늘 -> 리스트 변환
print(date_list)

url = 'https://search.naver.com/search.naver'

news_url_list = []

for date in date_list:
    # 해당 날짜를 문자열 형식으로 변환하여 검색 파라미터를 업데이트
    date_string = date.strftime('%Y.%m.%d') # 2023.08.01
    params['ds'] = date_string
    params['de'] = date_string
    params['nso'] = 'so:r,p:from{fr}to{to},a:all'.format(fr = date.strftime('%Y%m%d'), to = date.strftime('%Y%m%d'))
    time.sleep(1)

# 아래와 같은 형태(ds, de, nso)
# https://search.naver.com/search.naver?where=news&sm=tab_pge&query=AI
# &sort=2&photo=0&field=0&pd=3&ds=2023.08.01&de=2023.08.01&mynews=0&office_type=0&
# office_section_code=0&news_office_checked=&nso=so:r,p:from20230801to20230801,a:all&start=1
    
    res = requests.get(url, params=params)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')

    # a 태그중에 클래스가 info, text = 네이버뉴스인 모든 요소찾아 리스트로변환(네이버뉴스'로 연결되는 링크들이 포함)
    news_url_list_part = soup.find_all('a', 'info', text = '네이버뉴스')
    print(date_string, len(news_url_list_part), '개')
    news_url_list.extend(news_url_list_part)

In [5]:

# url을 parameter이용하여 가져온후 파싱을 거쳐 soup로 변환 
def get_soup(url, params):
    res = requests.get(url, params = params)
    time.sleep(1)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

#  주어진 BeautifulSoup 객체에서 '네이버뉴스' 링크를 모두 찾아서 반환
def get_news_url_list(soup):
    news_url_list = soup.find_all('a', 'info', text = '네이버뉴스')
    return news_url_list

def get_page_list(soup):
    # 클래스 이름이 'sc_page_inner'인 <div> 태그를 찾고,
    #  <div> 태그 내부의 모든 <a> 태그들을 찾아 각각의 텍스트를 정수로 변환
    page_list = list(map(lambda x: int(x.text),soup.find('div', 'sc_page_inner').find_all('a')))
    return page_list

def get_daily_news(url, params):
    news_url_list = []
    n = 1

    soup = get_soup(url, params)
    page_list = get_page_list(soup)

    while n in page_list:

        soup = get_soup(url, params)
        news_url_list_part = get_news_url_list(soup)
        news_url_list.extend(news_url_list_part)

        time.sleep(1)
        page_list = get_page_list(soup)
        print(n, page_list)
        
        n += 1
        params['start'] += 10

    print(len(news_url_list), '개,', n-1, '페이지')
    return news_url_list

In [ ]:
# 뉴스 URL 수집 | 하루
date = date_list[0]
print(date)
params['start'] = 1

date_string = date.strftime('%Y.%m.%d')
params['ds'] = date_string
params['de'] = date_string
params['nso'] = 'so:r,p:from{fr}to{to}'.format(fr = date.strftime('%Y%m%d'), to = date.strftime('%Y%m%d'))
time.sleep(1)

# print(date_string)
daily_news_list = get_daily_news(url, params)

In [ ]:
# (참고) 뉴스 URL 수집 | 모든 페이지
url = 'https://search.naver.com/search.naver'
params = parse_qs(urlparse(tmp_url).query)

news_url_list = []
for date in date_list:
    params['start'] = 1

    date_string = date.strftime('%Y.%m.%d')
    params['ds'] = date_string
    params['de'] = date_string
    params['nso'] = 'so:r,p:from{fr}to{to}'.format(fr = date.strftime('%Y%m%d'), to = date.strftime('%Y%m%d'))

    time.sleep(1)
    
    print(date_string)
    daily_news_list = get_daily_news(url, params)
    news_url_list.extend(daily_news_list)

# 1주일 분량에 46분 30초 소요

# 수집된 URL 개수 확인
len(daily_news_list)

# 뉴스 URL 추출
news_url_list = list(map(lambda x : x.get('href'), daily_news_list))

# for문을 이용하지 않고 뉴스 맵 함수를 이용하여 빠르게 URL을 추출합니다.
# URL 데이터 프레임으로 변환
df = pd.DataFrame(news_url_list, columns = ['URL']); print(df['URL'][61])

# URL 데이터 프레임 저장

path = ''
file_name = '네이버 뉴스_AI_{}.csv'.format(date_string)
df.to_csv(path + file_name, index = False)